### 데이터 설명

#### 1. train.jsonl - 학습에 사용 할 데이터셋
- media : 기사 미디어
- id : 각 데이터 고유 번호
- article_original : 전체 기사 내용, 문장별로 split되어 있음
- abstractive : 사람이 생성한 요약문
- extractive : 사람이 추출한 요약문 3개의 index

#### 2. abstractive_test.jsonl - 추론할 데이터셋
- media : 기사 미디어
- id : 각 데이터 고유 번호
- article_original : 전체 기사 내용, 문장별로 split되어 있음

#### 3. abstractive_sample_submission.csv - abstractive_test.jsonl의 추론 결과를 작성하여 제출
- id : abstractive_test.jsonl 데이터의 고유 번호
- summary : 모델이 추론한 문장

### 1. 패키지 호출

In [1]:
import pandas as pd
import numpy as np
import json
from summarizer import Summarizer
from transformers import *
import re

### 2. 데이터 가져오기

In [2]:
data_path= "C:/Users/User/Desktop/dacon_competition/data/ai_summarization/"
train_dt= pd.read_json(data_path+"train.json", lines=True, encoding='utf-8')
test_dt= pd.read_json(data_path+"abstractive_test_v2.json", lines=True, encoding='utf-8')
submission_dt= pd.read_csv(data_path+"abstractive_sample_submission_v2.csv")

In [3]:
train_dt.head()

,abstractive,article_original,extractive,id,media
0,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,"[0, 1, 2]",327827480,당진시대
1,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,"[0, 1, 3]",339840364,국제신문
2,16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아, 국유지로...","[6, 7, 10]",371071597,기호일보
3,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,[대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다...,"[1, 6, 4]",354806783,대구일보
4,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,[국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 ...,"[2, 3, 10]",347022773,대구신문


In [4]:
test_dt.head()

,article_original,id,media
0,[▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 ...,250366733,당진시대
1,"[신 벌떼해장국이 손님들의 성원에 보답하고자 24시간 영업을 재개한다., 또한 이와...",250371241,당진시대
2,"[1년 전과 비교해 당진지역 휘발유 가격은 72원, 경유 가격은 103원 상승했다....",250371242,당진시대
3,[어기구 국회의원이 천연가스의 안정적 수급을 위해 2020년 착공이 예정돼 있는 액...,252106905,당진시대
4,"[당진시정책자문위원회가 유명무실하다는 비판이 지속적으로 제기되고 있다., 민선6기가...",252639810,당진시대


In [5]:
submission_dt.head()

,id,summary
0,250366733,생성요약
1,250371241,생성요약
2,250371242,생성요약
3,252106905,생성요약
4,252639810,생성요약


In [6]:
len(train_dt)

42803

In [7]:
len(test_dt)

9987

In [8]:
# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('monologg/kobert')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('monologg/kobert')
custom_model = AutoModel.from_pretrained('monologg/kobert', config=custom_config)

In [9]:
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
#model=Summarizer()

In [10]:
original_text="""
사회적 거리두기 2단계 적용 지역에선 유흥시설 5종은 집합금지가 이뤄진다. 방문판매 등 직접판매홍보관은 4㎡에서 8㎡ 당 1명으로 인원 제한이 강화된다. 노래연습장은 오후 9시 이후 영업이 중단된다. 50㎡ 이상의 일반음식점, 휴게음식점, 제과점에 대해선 카페는 전 시간 포장·배달만 허용하고 음식점은 오후 9시 이후엔 식당 내 식사를 금지한다.
헬스장 등 실내체육시설은 오후 9시부터 운영을 중단한다. 놀이공원이나 워터파크는 수용 가능 인원의 3분의 1로 운영 규모를 줄인다. 국공립시설 중 경마 정륜 경정 카지노는 운영을 중단하고 이외 시설은 30% 이내로 제한한다. 스포츠 관람은 수용 가능 인원의 10%만 입장을 허용한다. 예배나 법회 등 종교활동도 좌석 수의 20%로 줄여야 한다."""

In [11]:
from summarizer import Summarizer

#model= Summarizer()
result= model(original_text)

In [12]:
result

'사회적 거리두기 2단계 적용 지역에선 유흥시설 5종은 집합금지가 이뤄진다. 놀이공원이나 워터파크는 수용 가능 인원의 3분의 1로 운영 규모를 줄인다.'

In [13]:
body= ''.join(test_dt['article_original'][2])

In [14]:
model(body)

''

In [15]:
body

'1년 전과 비교해 당진지역 휘발유 가격은 72원, 경유 가격은 103원 상승했다.지난해 1월 14일 기준으로 당진지역의 평균 휘발유 가격이 1418원, 경유 1197원이었던 반면, 올해에는 지난 4일 기준으로 휘발유 1490원, 경유 1300원인 것으로 조사됐다.한국석유공사가 운영하는 오피넷(www.opinet.co.kr)에 따르면 최근 송악읍 휘발유와 경유 평균 가격은 각각 1520원과 1323원으로 당진지역 14개 읍·면·동 가운데 가장 높은 것으로 나타났다.반면 면천면 휘발유 평균 가격은 1478원으로 가장 낮았으며, 경유는 정미면이 평균 가격 1254원으로 가장 저렴했다.더불어 당진시 전체 휘발유 평균 가격은 1490원으로 충남 휘발유 평균 가격인 1491원보다 1원 정도 낮았다.또한 경유 평균 가격은 1300원으로 충남 평균 1285원보다 15원 정도 높게 나타났다.주유소별 유류가격을 살펴보면 송악읍 (주)SK네트웍스가 휘발유와 경유가 각각 1747원과 1540원으로 가장 높았고 행담도주유소가 각각 1743원과 1540원으로 그 뒤를 이었다.한편 송악읍 (주)송악훼미리주유소의 휘발유 가격은 1439원으로 가장 낮았으며, 신평면 제성주유소의 경유 가격은 1237원으로 가장 저렴한 것으로 나타났다.유류가격은 읍·면·동별, 주유소별로 차이를 보이고 있는데, 여기에는 현대·SK 등 주유소 점유율이 높은 정유사들이 책정한 유류 판매가격이 한몫 하고 있다.또한 알뜰주유소의 경우 기름을 공급받는 곳을 유동적으로 변경하기 때문에 상대적으로 가격이 저렴하다.당진주유소 원종경 대표는 “OPEC(석유수출국기구)의 석유 감산 방침에 따라 국제유가가 전반적으로 상승했다”며 “이에 따라 국내 유가 또한 가파르게 상승하고 있다”고 말했다.형제주유소 오동원 대표는 “정유사가 국제유가를 공급가격에 반영하는 속도가 너무 빨라 주유소별 기름 판매가격이 상승하고 있다”고 최근 유가 상승 원인에 대해 설명했다.'

In [16]:
def cleasing(text):
    #pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail주소제거
    #text = re.sub(pattern=pattern, repl=' ', string=text)
    #pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    #text = re.sub(pattern=pattern, repl=' ', string=text)
    #pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 한글 자음, 모음 제거
    #text = re.sub(pattern=pattern, repl=' ', string=text)
    #pattern = '<[^>]*>'         # HTML 태그 제거
    #text = re.sub(pattern=pattern, repl=' ', string=text)
    #pattern = '[^\w\s]' # 특수기호제거
    #text = re.sub(pattern=pattern, repl=' ', string=text)
    
    #text = re.sub('[^.,a-zA-Z0-9 \n\.]', '', string=text)
    #text=text.replaceAll("[^.,a-zA-Z]")
    return text

In [17]:
test_dt= test_dt.reset_index()

In [18]:
result_concat= []

#for i in range(0,len(test_dt)):
for i in range(0,5):
    body= ''.join(test_dt['article_original'][i])
    body_cleansed= cleasing(body)
    result= model(body_cleansed)
    
    print(i)
    print("original")
    print(test_dt[test_dt['index']==i]['article_original'])
    print("body")
    print(body)
    print("body_cleansed")
    print(body_cleansed)
    print("result")
    print(result)
    
    if result=='':
        print("empty")
        result_concat.append(body)
    else:
        result_concat.append(result) 
    
#test_dt['result']= result_concat

0
original
0    [▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 ...
Name: article_original, dtype: object
body
▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓아두고 시위를 벌이고 있다.석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 30일 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓고 시위를 벌였다.43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당 2100원으로 책정된 임대료를 현재 쌀 판매가격인 1300원대로 인하할 것을 요구하고 있다.이들은 지난 12월 7일 농림축산식품부에 탄원서를 제출했지만 “임대료 인하는 올해 이후에나 가능하다”고 통보받은 상황이다.게다가 임차법인들의 계약기간이 올해 만료되기 때문에 임대료를 인하해도 지난 2년 동안의 손실 보상은 받을 수 없는 상황이다.이에 간척지협의회는 계약기간을 2년 연장하고, 연장된 기간 동안 인하된 임대료를 적용해 지난 2년 간의 손실에 대해 보상할 것을 제안했다.더불어 요구사항이 받아들여지지 않을 경우 벼 야적시위를 시작한 날짜인 지난해 12월 30일자로 임대료를 벼로 납부하겠다는 입장이다.김재용 봉치영농조합법인 조합원은 “현재 한국농어촌공사의 답변을 기다리고 있다”며 “상황을 지켜본 뒤 추가적인 야적 시위 여부를 결정할 계획”이라고 말했다.이어 “법을 만들고 집행할 때 현실성 있게 만들어야 한다”며 “농민이 정부를 믿을 수 있도록 조속히 해결책을 마련해주길 바란다”고 덧붙였다.
body_cleansed
▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓아두고 시위를 벌이고 있다.석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 30일 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓고 시위를 벌였다

In [19]:
test_dt.head()

,index,article_original,id,media
0,0,[▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 ...,250366733,당진시대
1,1,"[신 벌떼해장국이 손님들의 성원에 보답하고자 24시간 영업을 재개한다., 또한 이와...",250371241,당진시대
2,2,"[1년 전과 비교해 당진지역 휘발유 가격은 72원, 경유 가격은 103원 상승했다....",250371242,당진시대
3,3,[어기구 국회의원이 천연가스의 안정적 수급을 위해 2020년 착공이 예정돼 있는 액...,252106905,당진시대
4,4,"[당진시정책자문위원회가 유명무실하다는 비판이 지속적으로 제기되고 있다., 민선6기가...",252639810,당진시대


In [20]:
result= test_dt[['id','result']].rename({'result':'summary'},axis=1)

KeyError: "['result'] not in index"

In [ ]:
#result= test_dt[['id','article_original']].rename({'article_original':'summary'},axis=1)

In [ ]:
result.head()

In [ ]:
submission_dt['summary']=result['summary']

In [ ]:
submission_dt.head()

In [ ]:
#submission_dt.to_csv("submission_dt_1201.csv",encoding='cp949')
submission_dt.to_csv("submission_dt_1202.csv", index=False)

In [ ]:
submission_dt.info()

In [ ]:
result.info()

In [ ]:
submission_dt